In [1]:
from polars_ds.extensions import StrExt, NumExt
import polars as pl
import numpy as np 

In [ ]:
df = pl.DataFrame({
    "dummy": ["a"] * 50_000 + ["b"] * 50_000,
    "a": range(100_000),
    "b": range(-100_000, 0),
    "y": pl.Series(range(100_000, 200_000)) + 0.5
})

df.head()

In [5]:
df = pl.DataFrame({
    "y":[1,2,3,4,5],
    "a": [2,3,4,5,6],
    "b"
})

In [6]:
df.select(
    pl.col("y").num_ext.lstsq(pl.col("a"), add_bias=True)
)

y
list[f64]
"[1.0, -1.0]"


In [ ]:
df.group_by("dummy").agg(
    pl.col("y").num_ext.lstsq(pl.col("a"), pl.col("b"), add_bias = True)
)

In [ ]:
df.select(pl.col("y").num_ext.lstsq(pl.col("a"), pl.col("b")))

In [ ]:
df.select(
    pl.col("y").num_ext.lstsq(pl.col("a"), pl.col("b")).over("dummy")
)